南大os主页： http://jyywiki.cn/OS/2023/

jupyter: http://jyywiki.cn/OS/2023/build/lect2.ipynb

In [ ]:
计算机是一个状态机
cpu 是一个无情执行指令的机器
软件： os = 库函数（文件，进程，）
硬件： os = c 程序

## 1. 软件：什么是最小的os程序？

In [ ]:
示例程序 hello.c：
#include <stdio.h>
int main() {
    printf("hello world\n");
    return 0;
}

df@moss:~/g15/code/os/lect2$ gcc hello.c
df@moss:~/g15/code/os/lect2$ ls
a.out  hello.c

$ objdump -d a.out | less   # objdump 工具可以查看对应的汇编代码
$ objdump -d a.out | wc -l
$ gcc hello.c  -static  # -static 会链接 libc (大量的代码)

$ gcc hello.c  -static --verbose # 查看gcc所有编译选项
$ gcc  hello.c  -static -Wl,--verbose | less  # 可以查看所有链接选项
这样的程序不满足最小！！
计算机世界里没有任何魔法

## 2. 开始构造最小的应用程序



In [ ]:
示例程序 hello.c：

int main() {

}

df@moss:~/g15/code/os/lect2$ gcc -c hello.c
df@moss:~/g15/code/os/lect2$ objdump -d hello.o
0000000000000000 <main>:
   0:   f3 0f 1e fa             endbr64
   4:   55                      push   %rbp    # 
   5:   48 89 e5                mov    %rsp,%rbp # rsp--> rbp
   8:   b8 00 00 00 00          mov    $0x0,%eax  # eax 返回值
   d:   5d                      pop    %rbp
   e:   c3                      ret
解读： rbp, rsp 分别是栈底和栈顶指针

df@moss:~/g15/code/os/lect2$ ld hello.o -e main
df@moss:~/g15/code/os/lect2$ file a.out
a.out: ELF 64-bit LSB executable, x86-64, version 1 (SYSV), statically linked, not stripped
df@moss:~/g15/code/os/lect2$ ./a.out
Segmentation fault

In [ ]:
"""栈帧push的汇编解释,https://blog.csdn.net/qq_43325061/article/details/119066069#:~:text=push%20%25rbp%20%E6%98%AF%E6%8A%8A%20%25rbp,%E6%88%91%E7%9A%84%E6%A0%88%E5%BA%95%E4%BD%8D%E7%BD%AE%E3%80%82

下面是常见的反汇编代码

push   %rbp
mov    %rsp,%rbp
push   %rbx
sub    $0x18,%rsp

pop    %rbx
pop    %rbp
retq   

push %rbp是把%rbp寄存器的值保存到内存里面的数组模拟栈，结合下面的mov %rsp,%rbp可以知道，%rbp此刻push的就是上面一个栈帧的栈低位置，
同时mov也将本初始化为0长度的栈帧低保存起来，因为此刻rsp是上一个栈帧的栈顶位置，刚好就是我的栈底位置。
所以mov是把左边的操作数的值赋给右边。

下一步sub $0x18,%rsp表示将rsp减去24个字节，代表给传参留位置。这样rsp就开始移动。所以sub也是右边是结果，右边减去最左边。

push %rbx是因为是被调用者保护rbx

最后从栈弹出保存的rbx rbp到对应的寄存器，这样就相当于恢复的执行上下文。retq就返回到返回地址，一般是下一条指令地址。

"""


##  3.gdb调试，探究Segmentation fault 原因

In [ ]:
df@moss:~/g15/code/os/lect2$ gdb a.out
GNU gdb (Ubuntu 12.1-0ubuntu1~22.04) 12.1
....................
For help, type "help".
Type "apropos word" to search for commands related to "word"...
Reading symbols from a.out...
(No debugging symbols found in a.out)
(gdb) starti
Starting program: /mnt/c/Users/dufei/code/os/lect2/a.out

Program stopped.
0x0000000000401000 in main ()
(gdb)layout asm  # text UI
(gdb)si
(gdb)info registers  
(gdb)p $rsp
(gdb)x $rsp # 0x00000001
(gdb)si
0x0...0000001 in ?? ()

In [ ]:
那为何下面的程序不能正常执行呢？？, 使用 gdb 观测程序的执行
"""
int main() {
 
}
"""

gdb a.out
......
Type "apropos word" to search for commands related to "word"...
Reading symbols from a.out...(no debugging symbols found)...done.
(gdb) starti
Starting program: /code/daily-record/os/lect2/a.out 
warning: Error disabling address space randomization: Operation not permitted

Program stopped.
0x00000000004000b0 in main ()
(gdb) layout asm
(gdb) info reg
----------------
  │0x4000b4 <main+4>       mov    $0x0,%eax                                                                                       │
   │0x4000b9 <main+9>       pop    %rbp                                                                                            │
  >│0x4000ba <main+10>      retq                                                                                                   │
   │0x4000bb                add    %al,(%rax)                                                                                      │
   │0x4000bd                add    %al,(%rax)                                                                                      │
   │0x4000bf                add    %dl,(%rax,%rax,1)                                                                               │
   │0x4000c2                add    %al,(%rax)                                                                                      │
   │0x4000c4                add    %al,(%rax)                                                                                      │
   │0x4000c6                add    %al,(%rax)                                                                                      │
   │0x4000c8                add    %edi,0x52(%rdx)                                                                                 │
   │0x4000cb                add    %al,(%rcx)                                                                                      │
   │0x4000cd                js     0x4000df                                                                                        │
   │0x4000cf                add    %ebx,(%rbx)                                                                                     │
   │0x4000d1                or     $0x7,%al                                                                                        │
   │0x4000d3                or     %dl,0x1c000001(%rax)                                                                            │
   │0x4000d9                add    %al,(%rax)                                                                                      │
   │0x4000db                add    %bl,(%rax,%rax,1)                                                                               │
   │0x4000de                add    %al,(%rax)                                                                                      │
   │0x4000e0                sar    %bh                                                                                             │
   └───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘
native process 95639 In: main                                                                                    L??   PC: 0x4000ba 
rsp            0x7ffcec8e9200   0x7ffcec8e9200
r8             0x0      0
r9             0x0      0
r10            0x0      0
---Type <return> to continue, or q <return> to quit---q
Quit
(gdb) si
0x00000000004000b1 in main ()
0x00000000004000b4 in main ()
0x00000000004000b9 in main ()
0x00000000004000ba in main ()
(gdb) p $rsp
$1 = (void *) 0x7ffcec8e9200
(gdb) x $rsp
0x7ffcec8e9200: 0x00000001
(gdb) si
Cannot access memory at address 0x1  #  不合法这里
(gdb) 

In [ ]:
添加 死循环后可以正常执行：
"""
int main() {
  while (1); 
}
"""
root@3d972822392a:/code/daily-record/os/lect2# objdump -d a.out 

00000000004000b0 <main>:
  4000b0:       55                      push   %rbp
  4000b1:       48 89 e5                mov    %rsp,%rbp
  4000b4:       eb fe                   jmp    4000b4 <main+0x4>



In [ ]:
我们的计算机系统是状态机，os 提供 syscall()， 程序停止
最小的 hello world , minimal.S文件：


"""  这个程序不是很理解？？？
#include <sys/syscall.h>

.globl _start
_start:
  movq $SYS_write, %rax   // write(
  movq $1,         %rdi   //   fd=1,
  movq $st,        %rsi   //   buf=st,
  movq $(ed - st), %rdx   //   count=ed-st
  syscall                 // );

  movq $SYS_exit,  %rax   // exit(
  movq $1,         %rdi   //   status=1
  syscall                 // );

st:
  .ascii "\033[01;31mHello, OS World\033[0m\n"
ed:
"""

$ gcc -c minimal.S
$ ld minimal.o
$ ./a.out
Hello, OS World

## 4, c语言的状态机模型

In [ ]:
什么是程序？
gdb 解释器对照查看命令：
gcc hello.c -ggdb
gdb a.out
layout src
start


任何c语言代码--> 赋值，运算，if...goto，

这就是编译器

![image-20230409172821708](https://tva1.sinaimg.cn/large/008vOhrAgy1hctkrkr09tj30lq0dodgr.jpg)



## 5, 操作系统中 “任何程序” 的一生
任何程序 = minimal.S = 调用 syscall 的状态机

- 被操作系统加载
通过另一个进程执行 execve 设置为初始状态
- 状态机执行

    进程管理：fork, execve, exit, ...
    
    文件/设备管理：open, close, read, write, ...
    
    存储管理：mmap, brk, ...
- 调用 _exit (exit_group) 退出

e.g., 
![image-20230409213341415](https://tva1.sinaimg.cn/large/008vOhrAgy1hctruqa9h7j31io0bqmzj.jpg)

In [ ]:
# 查看 系统调用有哪些
man syscalls
man syscall

### 5.1, strace 打开程序的执行
将程序的所有系统调用打印出来

查看所有执行的系统调用系列

gcc demo:

将strace 输出以管道的方式给文本编辑器：

strace -f gcc hello.c |& vim -

:set nowrap # 折叠

:set nu

:%!grep execve  # 只查看 包含 execve的行

:%!grep -e execve -e open

![image-20230409221918348](https://tva1.sinaimg.cn/large/008vOhrAgy1hctt67irh9j31zu0l0k2q.jpg)